<a href="https://colab.research.google.com/github/Romanchenko-RS/ML/blob/main/%D0%94%D0%972_%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8_%D0%BF%D0%BE%D1%82%D0%B5%D1%80%D1%8C_%D0%B8_%D0%BE%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%A0%D0%BE%D0%BC%D0%B0%D0%BD%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%A0_%D0%A1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Самостоятельная реализация логистической регрессии**

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

In [ ]:
data = load_iris()
X = data.data[50:, :]  # Берём последние 100 записей, которые соответствуют классам 1 и 2
y = data.target[50:]
y[y == 2] = 0  # Преобразуем метки классов Iris Virginica в 0 (для бинарной классификации)

In [ ]:
# Добавляем служебный столбец 1 к X (для w0)
X = np.hstack([np.ones((X.shape[0], 1)), X]) # Нужен для корректной работы математической модели и учёта смещения

In [ ]:
# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Функция сигмоида
# Это функция активации, которая преобразует входное значение в вероятность от 0 до 1
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
# Функция потерь (бинарная кросс-энтропия)
def compute_loss(y, y_hat):
    m = y.shape[0]
    return -np.sum(y*np.log(y_hat) + (1-y)*np.log(1-y_hat)) / m

In [ ]:
# Функция для расчёта градиентов
def compute_gradients(X, y, y_hat):
    m = y.shape[0]
    return np.dot(X.T, (y_hat - y)) / m

In [ ]:
# Функция градиентного спуска для обучения модели
def gradient_descent(X, y, lr=0.01, epochs=1000):
    weights = np.zeros(X.shape[1])
    losses = []
    for i in range(epochs):
        y_hat = sigmoid(np.dot(X, weights))
        loss = compute_loss(y, y_hat)
        gradients = compute_gradients(X, y, y_hat)

        weights -= lr * gradients

        if i % 100 == 0:
            losses.append(loss)
            print(f'Epoch {i}, Loss: {loss:.4f}')

    return weights, losses

In [ ]:
# Обучение модели
start_time = time.time()
weights, losses = gradient_descent(X_train, y_train, lr=0.001, epochs=1000)
end_time = time.time()

Epoch 0, Loss: 0.6931
Epoch 100, Loss: 0.6804
Epoch 200, Loss: 0.6759
Epoch 300, Loss: 0.6715
Epoch 400, Loss: 0.6672
Epoch 500, Loss: 0.6630
Epoch 600, Loss: 0.6588
Epoch 700, Loss: 0.6546
Epoch 800, Loss: 0.6505
Epoch 900, Loss: 0.6465


In [ ]:
# Предсказание
predictions = sigmoid(np.dot(X_test, weights)) >= 0.5
accuracy = accuracy_score(y_test, predictions)
elapsed_time = end_time - start_time

In [ ]:
# Вывод результатов
print(f'Accuracy: {accuracy:.3f}')
print(f'Time taken: {elapsed_time:.2f} seconds')

Accuracy: 0.400
Time taken: 0.12 seconds


**Реализация метода градиентного спуска**

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
import pandas

In [ ]:
# Функция сигмоида
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
# Функция потерь
# Вычисляет значение функции потерь (кросс-энтропия), которое является мерой разности между предсказанными и настоящими метками классов
def loss(y, y_hat):
    return -np.mean(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))

In [ ]:
# Градиент
# Рассчитывает градиент функции потерь
def gradients(X, y, y_hat):
    m = X.shape[0]
    return np.dot(X.T, (y_hat - y)) / m

In [ ]:
# Функция расчёта градиента
def gradient_descent(X, y, lr=0.1, epochs=1000):
    weights = np.zeros(X.shape[1])  # Выбираем начальное значение весов
    for i in range(epochs):
        y_hat = sigmoid(np.dot(X, weights))  # С помощью текущих весов и функции сигмоиды рассчитывается предсказание модели
        loss_value = loss(y, y_hat)  # Вычисляется значение функции потерь между предсказанными значениями и настоящими метками классов
        grads = gradients(X, y, y_hat)  # Определяется градиент функции потерь в текущей точке
        weights -= lr * grads  # Веса обновляются в направлении анти-градиента с учетом коэффициента скорости обучения
        if i % 100 == 0:
            print(f'Epoch {i}, Loss: {loss_value}')  # Следим за значением функции потерь с шагов в 100 итераций
    return weights

In [ ]:
# Функция расчёта rmsprop
def gradient_descent_rmsprop(X, y, lr=0.01, beta=0.9, epsilon=1e-8, epochs=1000):
    weights = np.zeros(X.shape[1])
    S = np.zeros(X.shape[1])
    for i in range(epochs):
        y_hat = sigmoid(np.dot(X, weights))
        grads = gradients(X, y, y_hat)
        S = beta * S + (1 - beta) * np.square(grads)
        weights -= lr * grads / (np.sqrt(S) + epsilon)
        if i % 100 == 0:
            loss_value = loss(y, y_hat)
            print(f'Epoch {i}, Loss: {loss_value}')
    return weights

In [ ]:
# Функция расчёта nadam
def nadam(X, y, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-7, epochs=1000):
    weights = np.zeros(X.shape[1])
    m = np.zeros(X.shape[1])
    v = np.zeros(X.shape[1])
    m_hat = np.zeros(X.shape[1])  # Корректированный момент первого порядка
    v_hat = np.zeros(X.shape[1])  # Корректированный момент второго порядка

    for i in range(epochs):
        y_hat = sigmoid(np.dot(X, weights))
        grads = gradients(X, y, y_hat)
        m = beta1 * m + (1 - beta1) * grads
        v = beta2 * v + (1 - beta2) * (grads ** 2)

        m_hat = m / (1 - beta1 ** (i + 1))
        v_hat = v / (1 - beta2 ** (i + 1))

        m_bar = (1 - beta1) * grads + beta1 * m_hat  # Коррекция Нестерова

        weights -= lr * m_bar / (np.sqrt(v_hat) + epsilon)

        if i % 100 == 0:
            loss_value = loss(y, y_hat)
            print(f'Epoch {i}, Loss: {loss_value}')

    return weights

In [ ]:
# Создание словаря для записи результатов
results = pandas.DataFrame(columns=['Method', 'Accuracy', 'Time'])

In [ ]:
# Расчёт градиентного спуска
start_time_gd = time.time()
weights = gradient_descent(X_train, y_train, lr=0.1, epochs=1000)
predictions_gd = sigmoid(np.dot(X_test, weights)) >= 0.5
accuracy_gd = accuracy_score(y_test, predictions_gd)
end_time_gd = time.time()
elapsed_time_gd = end_time_gd - start_time_gd
print(f'Accuracy gd: {accuracy_gd:.3f}')
print(f'Время выполнения gd: {elapsed_time_gd:.3f} секунд')

Epoch 0, Loss: 0.6931471805599453
Epoch 100, Loss: 0.411573398932988
Epoch 200, Loss: 0.3021388886021
Epoch 300, Loss: 0.24513904509931744
Epoch 400, Loss: 0.21005358925069723
Epoch 500, Loss: 0.186083946381474
Epoch 600, Loss: 0.168529713690611
Epoch 700, Loss: 0.15502643144676914
Epoch 800, Loss: 0.14425463820243017
Epoch 900, Loss: 0.13541874915791174
Accuracy gd: 0.850
Время выполнения gd: 0.058 секунд


In [ ]:
# Расчёт rmsprop
start_time_rmsprop = time.time()
weights_rmsprop = gradient_descent_rmsprop(X_train, y_train, lr=0.01, epochs=1000)
predictions_rmsprop = sigmoid(np.dot(X_test, weights_rmsprop)) >= 0.5
accuracy_rmsprop = accuracy_score(y_test, predictions_rmsprop)
end_time_rmsprop = time.time()
elapsed_time_rmsprop = end_time_rmsprop - start_time_rmsprop
print(f'Accuracy rmsprop: {accuracy_rmsprop:.3f}')
print(f'Время выполнения rmsprop: {elapsed_time_rmsprop:.3f} секунд')

Epoch 0, Loss: 0.6931471805599453
Epoch 100, Loss: 0.4015033666910816
Epoch 200, Loss: 0.2575211368662632
Epoch 300, Loss: 0.17917539949597372
Epoch 400, Loss: 0.13318359687233675
Epoch 500, Loss: 0.10413036941404223
Epoch 600, Loss: 0.08465199460042201
Epoch 700, Loss: 0.07094712441228172
Epoch 800, Loss: 0.06091002608874742
Epoch 900, Loss: 0.0533074551294178
Accuracy rmsprop: 0.850
Время выполнения rmsprop: 0.040 секунд


In [ ]:
# Расчёт nadam
start_time_nadam = time.time()
weights_nadam = nadam(X_train, y_train, lr=0.001, epochs=1000)
predictions_nadam = sigmoid(np.dot(X_test, weights_nadam)) >= 0.5
accuracy_nadam = accuracy_score(y_test, predictions_nadam)
end_time_nadam = time.time()
elapsed_time_nadam = end_time_nadam - start_time_nadam
print(f'Accuracy Nadam: {accuracy_nadam:.3f}')
print(f'Время выполнения Nadam: {elapsed_time_nadam:.3f} секунд')

Epoch 0, Loss: 0.6931471805599453
Epoch 100, Loss: 0.6559515943097727
Epoch 200, Loss: 0.6217524588538285
Epoch 300, Loss: 0.5890773242287743
Epoch 400, Loss: 0.5584090824407066
Epoch 500, Loss: 0.529801221978087
Epoch 600, Loss: 0.503184209056647
Epoch 700, Loss: 0.478444141948712
Epoch 800, Loss: 0.4554516015857522
Epoch 900, Loss: 0.4340744490556913
Accuracy Nadam: 0.850
Время выполнения Nadam: 0.053 секунд


In [ ]:
# Запись результатов в словарь
results.loc[len(results)] = ['MY_LR', accuracy, elapsed_time]
results.loc[len(results)] = ['GD', accuracy_gd, elapsed_time_gd]
results.loc[len(results)] = ['rmsprop', accuracy_rmsprop, elapsed_time_rmsprop]
results.loc[len(results)] = ['Nadam', accuracy_nadam, elapsed_time_nadam]

In [ ]:
# Вывод результатов
print(results)

    Method  Accuracy      Time
0    MY_LR      0.40  0.124074
1       GD      0.85  0.058236
2  rmsprop      0.85  0.040069
3    Nadam      0.85  0.052810


**Дополнительный расчёт из стандартной библиотеки**

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
# Выполним стандартизацию данных
model = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=1000)  # кол-во итераций для обучения
    )

In [ ]:
# Выполним обучение модели
start_time_box_lr = time.time()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
end_time_box_lr = time.time()
elapsed_time_box_lr = end_time_box_lr - start_time_box_lr
print(f'Время выполнения готового решения по регрессии: {elapsed_time_nadam:.3f} секунд')

Время выполнения готового решения по регрессии: 0.053 секунд


In [ ]:
accuracy_box_lr = accuracy_score(y_test, predictions)
print(f"accuracy для логистической регрессии: {test_score_lr:.3f} ")

accuracy для логистической регрессии: 0.800 


In [ ]:
results.loc[len(results)] = ['BOX_LR', accuracy_box_lr, elapsed_time_box_lr]

In [ ]:
print(results)

    Method  Accuracy      Time
0    MY_LR      0.40  0.124074
1       GD      0.85  0.058236
2  rmsprop      0.85  0.040069
3    Nadam      0.85  0.052810
4   BOX_LR      0.80  0.009026


**Выводы**

Для каждого метода выбрал значение итераций для обучения - 1000.
При таком количестве итераций лучше всего себя показала rmsprop.
Общая сравнительная картина представлена выше в таблице.

На мой взгялд, данная выборка мала для обучения модели, однако неплохо себя показывает, даже при использовании стандартного готового метода LogisticRegression.

Также видно, что за счёт стандартизации данных время выполнения стандартного решения становится наименьшим среди представленных методов.